In [2]:
import pandas as pd
import numpy as np
import sys

# ----------------------------------------------------
# ⬇️⬇️⬇️ 請修改這裡 ⬇️⬇️⬇️
# ----------------------------------------------------

# 檔案路徑 (請修改成你的檔案名稱)
file1_path = "predictions.csv"  # 你的第一個檔案 (例如 Server 跑出來的)
file2_path = "submission.csv" # 你的第二個檔案 (例如 要上傳的)

# 要用哪個欄位當作 Key 來比對
KEY_COLUMN = "name"

# 要比較數值的欄位
VALUE_COLUMN = "price"

# ----------------------------------------------------
# ⬆️⬆️⬆️ 修改完畢，請直接執行此儲存格 ⬆️⬆️⬆️
# ----------------------------------------------------

print("--- 程式開始執行 ---")

try:
    # --- 1. 載入 CSV 檔案 ---
    df1 = pd.read_csv(file1_path)
    df2 = pd.read_csv(file2_path)
    
    print(f"✅ 成功載入 File 1: {file1_path} ({len(df1)} 筆資料)")
    display(df1.head())
    
    print(f"✅ 成功載入 File 2: {file2_path} ({len(df2)} 筆資料)")
    display(df2.head())

    # --- 2. 快速檢查 (100% 相同) ---
    print("\n--- 1. 快速檢查 (是否 100% 相同) ---")
    if df1.equals(df2):
        print("✅ 檔案 100% 相同 (包含順序).")
        # 如果 100% 相同，就不需要往下比了
    else:
        print("ℹ️ 檔案內容或順序不完全相同，將進行詳細比較。")

        # --- 3. 合併資料以進行比對 ---
        print(f"\n--- 2. 使用 '{KEY_COLUMN}' 進行詳細比較 ---")
        val_f1 = f"{VALUE_COLUMN}_f1"
        val_f2 = f"{VALUE_COLUMN}_f2"

        merged_df = pd.merge(
            df1,
            df2,
            on=KEY_COLUMN,
            how='outer',
            suffixes=('_f1', '_f2') # 幫 price 欄位加上後綴
        )

        # --- 4. 找出差異 ---
        
        # 4.1 只存在於 File 1 (File 2 的 price 是 NaN)
        unique_to_f1 = merged_df[merged_df[val_f2].isnull()]

        # 4.2 只存在於 File 2 (File 1 的 price 是 NaN)
        unique_to_f2 = merged_df[merged_df[val_f1].isnull()]

        # 4.3 兩邊都存在，但 price 數值不同
        common_rows = merged_df.dropna() # 只看兩邊都存在的
        
        if not common_rows.empty:
            diff_mask = ~np.isclose(common_rows[val_f1], common_rows[val_f2])
            value_differences = common_rows[diff_mask]
        else:
            value_differences = pd.DataFrame() # 建立空 DF

        # --- 5. 輸出總結報告 ---
        print("\n--- 3. 差異總結 ---")
        total_diff = len(unique_to_f1) + len(unique_to_f2) + len(value_differences)
        
        if not unique_to_f1.empty:
            print(f"🔴 {len(unique_to_f1)} 筆資料只存在於 {file1_path} (File 1):")
            display(unique_to_f1[[KEY_COLUMN, val_f1]].head())
        
        if not unique_to_f2.empty:
            print(f"🔴 {len(unique_to_f2)} 筆資料只存在於 {file2_path} (File 2):")
            display(unique_to_f2[[KEY_COLUMN, val_f2]].head())

        if not value_differences.empty:
            print(f"🔴 {len(value_differences)} 筆資料的 '{VALUE_COLUMN}' 數值不同:")
            display(value_differences[[KEY_COLUMN, val_f1, val_f2]].head())
        
        if total_diff == 0:
            print("✅ 檔案內容相同 (但資料順序可能不同).")
        else:
            print(f"\n🔴 總共有 {total_diff} 筆資料不符。")


except FileNotFoundError as e:
    print(f"🔴 錯誤: 找不到檔案. 請檢查你的 file1_path 和 file2_path 設定。")
    print(f"詳細錯誤: {e}", file=sys.stderr)
except Exception as e:
    print(f"🔴 處理過程中發生未預期的錯誤: {e}", file=sys.stderr)

print("\n--- 程式執行完畢 ---")

--- 程式開始執行 ---
✅ 成功載入 File 1: predictions.csv (55043 筆資料)


,name,price
0,!(呷)蛋黃粽-熱,113.925348
1,#13-PILOT可擦印章 FRIXION stamp,65.011854
2,#17-PILOT可擦印章 FRIXION stamp,65.011854
3,#304隔熱湯碗組16CM,295.490612
4,#316方形筷盒組-20cm,322.783908


✅ 成功載入 File 2: submission.csv (55043 筆資料)


,name,price
0,!(呷)蛋黃粽-熱,113.925348
1,#13-PILOT可擦印章 FRIXION stamp,65.011854
2,#17-PILOT可擦印章 FRIXION stamp,65.011854
3,#304隔熱湯碗組16CM,295.490612
4,#316方形筷盒組-20cm,322.783908



--- 1. 快速檢查 (是否 100% 相同) ---
ℹ️ 檔案內容或順序不完全相同，將進行詳細比較。

--- 2. 使用 'name' 進行詳細比較 ---

--- 3. 差異總結 ---
🔴 5 筆資料的 'price' 數值不同:


,name,price_f1,price_f2
961,5in1腦力大挑戰桌遊,526.188145,520.435879
2508,Canon 4x6相片紙含色帶*108張(KP-108IN),1030.583710,994.637296
13652,【Canon】KP-108IN(4x6相片紙含色帶-1入 108張),794.364841,806.218420
32350,【彩之舞】噴墨RC亮面高畫質數位相紙-防水 270g 6×8in 20張/包 HY-B68(...,276.732356,272.412079
53896,雙效起司潔牙棒M桶裝[YS-118-331000GB],1154.611540,1100.758203



🔴 總共有 5 筆資料不符。

--- 程式執行完畢 ---
